iMPORTS

In [1]:
import json
import requests
from bs4 import BeautifulSoup

In [2]:
with open("./cookie.json","r",encoding="UTF-8") as jf:
    headers = json.load(jf)

1. send a request for accessing first webpage with opinions about products


In [3]:
url = "https://www.ceneo.pl/169771365#tab=reviews"
response = requests.get(url, headers = headers)

2. If responce is ok, parse HTML page content into DOM structure 

In [4]:
if response.status_code == 200:
    page_dom = BeautifulSoup(response.text, 'html.parser')
    

3. Extract from DOM structure  opinions and their components

In [ ]:
opinions = page_dom.select("div.user-post user-post__card js_product-review:not(.user-post--highlights)")
print(type(page_dom))
print(type(opinions))
print(len(opinions))
opinions = opinions.pop(0)
print(type(opinions))